In [1]:
import pandas as pd
import numpy as np
import json 
from CounterSpeechGenerator import process_train_data

Setting up an example: list of hate speech, list of "gpt2" counterspeech, list of "gpt3" counterspeech (not actually, just pretending)

In [4]:
hs_cs_pairs, targets = process_train_data()

In [5]:
hatespeech = [hs for (hs, _, _) in hs_cs_pairs]
counterspeech = [cs for (_, cs, _) in hs_cs_pairs]

In [7]:
hatespeech = np.array(hatespeech)

In [8]:
unique, counts = np.unique(hatespeech, return_counts=True)
duplicates = unique[counts > 1]

In [10]:
duplicates = duplicates[:5]

In [22]:
hatespeeches = []
target_labels = []
gpt2_counterspeech = []
gpt3_counterspeech = []

In [23]:
for hs in duplicates:
    indices = np.where(hatespeech == hs)[0]
    cs1 = counterspeech[indices[0]]
    cs2 = counterspeech[indices[1]]
    target_labels.append(targets[indices[0]])
    hatespeeches.append(hs)
    gpt2_counterspeech.append(cs1)
    gpt3_counterspeech.append(cs2)

In [24]:
hatespeeches

['99% of Muslims in the UK are unemployable and refuse to work.',
 "A Muslim's only real allegiance is to Allah (as a comment in relation to election for public office etc.)",
 'According to the Bible, then gays are an abomination to God.',
 'Afghanistan or maybe Pakistan? I bet this does not look like an English town to you? #saville Town (Text accompanying video footage of people on their way to Friday prayers etc. )',
 'Africans are the cause of all the ills in the world. Plus, their bodies are weak and we are born to dominate them.']

In [25]:
target_labels

['MUSLIMS', 'MUSLIMS', 'LGBT+', 'MUSLIMS', 'POC']

In [26]:
gpt2_counterspeech

['That is a blatant lie. (attach unemployment statistics)',
 'So their faith and belief in the teachings of God guides them to a virtuous path of public service. What exactly are you complaining about?',
 'The Bible clearly teaches that everyone is born equal in dignity with the Lord.',
 'Small groups of people peacefully gather to pray, but because they do not look like you, you are outraged? Why am I not surprised?',
 "Aree you suggesting that we shouldn't live with Africans? To think that we are genetically superior to them because of their physical characteristics is just wrong."]

In [27]:
gpt3_counterspeech

['The vast majority of them bring necessary skills into our economy/society. Plus, they are eager to work. I would suggest to check your sources?',
 'So you are saying that we should only allow non-religious people into public office?',
 "Homosexuality has been practiced throughout history in all major civilizations. It is just that today's people accept homosexuality as normal and accept people of other backgrounds too.",
 'Omg brown people contributing to our society. How dare they!?',
 'Well, the argument sounds like you are trying to justify racial hatred by making some generalisations. Is that the case?']

Using reddit/tumblr data:

In [22]:
with open('Data/tumblr_gpt2_counterspeech.json', 'r') as f:
    gpt2_data = json.load(f)
with open('Data/tumblr_gpt3_counterspeech.json', 'r') as f:
    gpt3_data = json.load(f)

In [23]:
hatespeeches = []
target_labels = []
gpt2_counterspeech = []
gpt3_counterspeech = []

for i in gpt2_data:
    hatespeeches.append(gpt2_data[i]['hatespeech'])
    target_labels.append(gpt2_data[i]['target'])
    gpt2_counterspeech.append(gpt2_data[i]['counterspeech'])

for i in gpt3_data:
    gpt3_counterspeech.append(gpt3_data[i]['counterspeech'])

Creating a df comparing the generated responses:

In [24]:
data = {}
for i, hs in enumerate(hatespeeches):
    target = target_labels[i]
    if np.random.random() < 0.5:
        model1_cs = gpt2_counterspeech[i]
        model2_cs = gpt3_counterspeech[i]
        model1_is_gpt2 = True
    else:
        model2_cs = gpt2_counterspeech[i]
        model1_cs = gpt3_counterspeech[i]
        model1_is_gpt2 = False
    data[i] = {
        'Hatespeech': hs,
        'Target': target,
        'Model 1 CS': model1_cs,
        'Model 2 CS': model2_cs,
        'Model 1 is GPT2': model1_is_gpt2
    }

In [25]:
data

{0: {'Hatespeech': 'oloradoron: Is it really an insult to have relics of the past? Should Egypt tear down the pyramids because they were built with slave labor? History is replete with almost every culture and nation having slaves when they were the victors, or the strongest. Slavery is not unique to Africans, nor unique to the United States. For centuries Muslim countries on the Mediterranean Sea dealt in huge slave trades. Many of those slaves were white. Conquered nations routinely were turned into slaves. It is a very ugly and shameful history of most of humanity. Ugly or not it shouldn’t be swept under the carpet and forgotten. That is how history repeats itself. I understand why Confederate statues can be construed as offensive. They represent a sordid and brutal past that no American today should think well of. There should be no honoring of the institution of slavery, ever. The Civil War was not just about slavery. It was about a country being torn apart by a divisive and growi

In [26]:
df = pd.DataFrame(data)

In [27]:
df = df.T

Write to excel file:

In [28]:
df.to_excel('Data/gpt_tumblr_comparison.xlsx')

Loading excel file, bernoulli test:

In [40]:
df = pd.read_excel('Gpt2 vs 3 Comparison.xlsx', index_col=0, engine='openpyxl')

In [42]:
df

,Hatespeech,Target,Model 1 CS,Model 2 CS,Model 1 is GPT2,Syntax/Grammar,Talks about correct target,Addresses concepts in the hatespeech
0,99% of Muslims in the UK are unemployable and ...,MUSLIMS,The vast majority of them bring necessary skil...,That is a blatant lie. (attach unemployment st...,False,1,1,1
1,A Muslim's only real allegiance is to Allah (a...,MUSLIMS,So their faith and belief in the teachings of ...,So you are saying that we should only allow no...,True,1,2,1
2,"According to the Bible, then gays are an abomi...",LGBT+,Homosexuality has been practiced throughout hi...,The Bible clearly teaches that everyone is bor...,False,1,1,2
3,Afghanistan or maybe Pakistan? I bet this does...,MUSLIMS,Omg brown people contributing to our society. ...,Small groups of people peacefully gather to pr...,False,2,2,1
4,Africans are the cause of all the ills in the ...,POC,"Well, the argument sounds like you are trying ...",Aree you suggesting that we shouldn't live wit...,False,1,1,1


In [47]:
df.columns

Index(['Hatespeech', 'Target', 'Model 1 CS', 'Model 2 CS', 'Model 1 is GPT2',
       'Syntax/Grammar', 'Talks about correct target',
       'Addresses concepts in the hatespeech'],
      dtype='object')

In [43]:
data = df.T

In [46]:
data = data.to_dict()

In [50]:
data

{0: {'Hatespeech': '99% of Muslims in the UK are unemployable and refuse to work.',
  'Target': 'MUSLIMS',
  'Model 1 CS': 'The vast majority of them bring necessary skills into our economy/society. Plus, they are eager to work. I would suggest to check your sources?',
  'Model 2 CS': 'That is a blatant lie. (attach unemployment statistics)',
  'Model 1 is GPT2': False,
  'Syntax/Grammar': 1,
  'Talks about correct target': 1,
  'Addresses concepts in the hatespeech': 1},
 1: {'Hatespeech': "A Muslim's only real allegiance is to Allah (as a comment in relation to election for public office etc.)",
  'Target': 'MUSLIMS',
  'Model 1 CS': 'So their faith and belief in the teachings of God guides them to a virtuous path of public service. What exactly are you complaining about?',
  'Model 2 CS': 'So you are saying that we should only allow non-religious people into public office?',
  'Model 1 is GPT2': True,
  'Syntax/Grammar': 1,
  'Talks about correct target': 2,
  'Addresses concepts in

Un-anonymize

In [53]:
new_data = {}
col_syntax = 'Syntax/Grammar'
col_correct_target = 'Talks about correct target'
col_correct_concept = 'Addresses concepts in the hatespeech'
for i in data:
    hs = data[i]['Hatespeech']
    target = data[i]['Target']

    model1_is_gpt2 = data[i]['Model 1 is GPT2']
    if model1_is_gpt2:
        gpt2_counterspeech = data[i]['Model 1 CS']
        gpt3_counterspeech = data[i]['Model 2 CS']
        syntax = 'GPT2' if data[i][col_syntax] == 1 else 'GPT3'
        correct_target = 'GPT2' if data[i][col_correct_target] == 1 else 'GPT3'
        correct_concept = 'GPT2' if data[i][col_correct_concept] == 1 else 'GPT3'
    else:
        gpt2_counterspeech = data[i]['Model 2 CS']
        gpt3_counterspeech = data[i]['Model 1 CS']  
        syntax = 'GPT2' if data[i][col_syntax] == 2 else 'GPT3'
        correct_target = 'GPT2' if data[i][col_correct_target] == 2 else 'GPT3'
        correct_concept = 'GPT2' if data[i][col_correct_concept] == 2 else 'GPT3'
       

    new_data[i] = {
        'Hatespeech': hs,
        'Target': target,
        'GPT2 CS': gpt2_counterspeech,
        'GPT3 CS': gpt3_counterspeech,
        col_syntax: syntax,
        col_correct_target: correct_target,
        col_correct_concept: correct_concept
    }   

In [ ]:
new_data

In [61]:
new_df = pd.DataFrame(new_data).T

In [62]:
new_df

,Hatespeech,Target,GPT2 CS,GPT3 CS,Syntax/Grammar,Talks about correct target,Addresses concepts in the hatespeech
0,99% of Muslims in the UK are unemployable and ...,MUSLIMS,That is a blatant lie. (attach unemployment st...,The vast majority of them bring necessary skil...,GPT3,GPT3,GPT3
1,A Muslim's only real allegiance is to Allah (a...,MUSLIMS,So their faith and belief in the teachings of ...,So you are saying that we should only allow no...,GPT2,GPT3,GPT2
2,"According to the Bible, then gays are an abomi...",LGBT+,The Bible clearly teaches that everyone is bor...,Homosexuality has been practiced throughout hi...,GPT3,GPT3,GPT2
3,Afghanistan or maybe Pakistan? I bet this does...,MUSLIMS,Small groups of people peacefully gather to pr...,Omg brown people contributing to our society. ...,GPT2,GPT2,GPT3
4,Africans are the cause of all the ills in the ...,POC,Aree you suggesting that we shouldn't live wit...,"Well, the argument sounds like you are trying ...",GPT3,GPT3,GPT3


In [57]:
from scipy.stats import binom_test

In [81]:
n_trials = len(new_df)
n_success_syntax = new_df[col_syntax].value_counts()['GPT3']
n_success_target = new_df[col_correct_target].value_counts()['GPT3']
n_success_concept = new_df[col_correct_concept].value_counts()['GPT3']

In [82]:
syntax_p = binom_test(n_success_syntax, n_trials)
target_p = binom_test(n_success_target, n_trials)
concept_p = binom_test(n_success_concept, n_trials)

In [83]:
print(syntax_p)
print(target_p)
print(concept_p)

1.0
0.375
1.0
